## Configuration

Paste the correct values in the cell below. The **url** should be the URL for the cluster. Do **NOT** include a trailing "/" character, or the "zen" portion of the URL from the Cloud Pak for Data dashboard. For the user credentials, the user should have admin authority for the OpenScale service, as the notebook will be adding machine learning providers to OpenScale.

In [ ]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud",
    "username": "admin",
    "password": "xxxxxxxxx",
    "version": "3.5"
}

This notebook will remove the deployment space from OpenScale's list of machine learning providers. It is highly recommended that you create a new deployment space, as any other models deployed to the same space will be removed from OpenScale monitoring. Deployment spaces can be managed from the [Deployments screen](../../../../../ml-runtime/spaces?context=icp4data).

In [ ]:
WML_PRODUCTION_SPACE = 'xxxxxx'

Paste the OpenScale datamart ID into the cell below. It can be found by navigating to the [OpenScale dashboard](../../../../../aiopenscale/insights?serviceInstanceNamespace=zen&serviceInstanceDisplayName=OpenScale), navigating to the **Model information** view of any model, and scrolling to the **Integration details** section.

In [ ]:
DATAMART_ID = '00000000-0000-0000-0000-1616425950709603'

Paste the name of the deployed models below. It is case-sensitive.

In [ ]:
PRODUCTION_DEPLOYMENT = 'predictive attrition - 2018'

## Run the notebook

At this point, you may run all remaining cells of the notebook. When it completes, you can return to the OpenScale dashboard to see the results.

## Authenticate clients

In [ ]:
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [ ]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

In [ ]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

Errors in this next cell are typically caused by using the incorrect URL in the first configuration cell. Ensure that you did not end the URL with a trailing "/" character or include the "zen" portion of the Cloud Pak for Data dashboard URL.

In [ ]:
authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )

wos_client = APIClient(service_url=WOS_CREDENTIALS['url'], service_instance_id=DATAMART_ID, authenticator=authenticator)
wos_client.version

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

## Remove existing spaces as OpenScale machine learning providers

In [ ]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_id = service_provider.entity.deployment_space_id
    if service_instance_id == WML_PRODUCTION_SPACE or service_instance_id == WML_PREPROD_SPACE:
        wos_client.service_providers.delete(service_provider.metadata.id)
        print("Deleted existing service_provider for WML instance: {}".format(service_instance_id))

## Promote 2018 to deployment space and deploy

## Add production deployment space as OpenScale service provider

In [ ]:
SERVICE_PROVIDER_NAME = "Attrition Production Space"
SERVICE_PROVIDER_DESCRIPTION = "Production space for the HR attrition model"

In [ ]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_PRODUCTION_SPACE,
        operational_space_id = "production",
        credentials=WML_CREDENTIALS,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id

In [ ]:
wml_client.set.default_space(WML_PRODUCTION_SPACE)

In [ ]:
DEPLOYMENT_UID = ''
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    if deployment['metadata']['name'] == PRODUCTION_DEPLOYMENT:
        DEPLOYMENT_UID = deployment['metadata']['id']
print(DEPLOYMENT_UID)

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=DATAMART_ID, service_provider_id=service_provider_id, deployment_id=DEPLOYMENT_UID, deployment_space_id = WML_PRODUCTION_SPACE).result['resources'][0]
asset_deployment_details

In [ ]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=DATAMART_ID,service_provider_id=service_provider_id,deployment_id=DEPLOYMENT_UID,deployment_space_id=WML_PRODUCTION_SPACE)

In [ ]:
feature_columns = ['POSITION_CODE','DEPARTMENT_CODE','DAYS_WITH_COMPANY','COMMUTE_TIME','AGE_BEGIN_PERIOD','GENDER_CODE','PERIOD_TOTAL_DAYS','STARTING_SALARY','ENDING_SALARY','NB_INCREASES','BONUS','NB_BONUS','VACATION_DAYS_TAKEN','SICK_DAYS_TAKEN','PROMOTIONS','NB_MANAGERS','DAYS_IN_POSITION','DAYS_SINCE_LAST_RAISE','RANKING_CODE','OVERTIME','DBLOVERTIME','TRAVEL']
cat_features = []

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATAMART_ID,
        service_provider_id=service_provider_id,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column="ATTRITION",
            probability_fields=["probability"],
            prediction_field="prediction",
            feature_fields = feature_columns,
            categorical_fields = cat_features,
            training_data_reference=TrainingDataReference(type="cos",
                                                          location=COSTrainingDataReferenceLocation(bucket = "faststartlab-donotdelete-pr-nhfd4jnhlxgpc7",
                                                                                                    file_name = "hr_attrition_data_2019.csv"),
                                                          connection=COSTrainingDataReferenceConnection.from_dict({
                                                                        "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/7d8b3c34272c0980d973d3e40be9e9d2:2883ef10-23f1-4592-8582-2f2ef4973639::",
                                                                        "url": "https://s3.us.cloud-object-storage.appdomain.cloud",
                                                                        "api_key": "yqcPbWZ0AQPHleHVerrR4Wx5e9pymBdMgydbEra5zCif",
                                                                        "iam_url": "https://iam.ng.bluemix.net/oidc/token"}))
        )
    ).result
subscription_id = subscription_details.metadata.id
print('subscription_id: ' + subscription_id)

In [ ]:
scoring_data = {
    "input_data": [
        {"fields": feature_columns,
         "values": [
             [3900.0,300.0,60.0,12.0,38.0,0.0,60.0,27500.0,27500.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,60.0,60.0,3.0,0.0,0.0,0.0]
         ]
        }
    ]
}

In [ ]:
import time

time.sleep(5)
prediction = wml_client.deployments.score(DEPLOYMENT_UID, scoring_data)
prediction

In [ ]:
time.sleep(5)

payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

In [ ]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

In [ ]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)


target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

In [ ]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds 
).result
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATAMART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

## Feed in historical data